<a href="https://colab.research.google.com/github/PoChihKuo/RECA-CXR/blob/master/Test_and_report_metrics_mimic_cxr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x 

In [ ]:
#Model-MMC
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/mimic-cxr-models/bfantasykuo_gmail_com/model_mimic_gen_aug_epoch5.h5' 

#Model-RECA-MMC
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/mimic-cxr-models/bfantasykuo_gmail_com/model_aug_v7_combine_u0w1_bach32_epoch5.h5' # used in paper

#Model-PHO-MMC
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/mimic-cxr-models/bfantasykuo_gmail_com/model_mimic_photo_epoch50.h5' 

#Model-TRA-MMC
PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/bfantasykuo_gmail_com/model_MMC_transfer_test.h5'
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/bfantasykuo_gmail_com/model_CXP_transfer_new.h5'

# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/mimic-cxr-models/bfantasykuo_gmail_com/model_mimic_v7-rand_epoch5.h5' 
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/mimic-cxr-models/bfantasykuo_gmail_com/model_mimic_v7-man_epoch5.h5' 


#Model-CXP
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/chexpert-cxr-models/bfantasykuo_gmail_com/model_chexpert_gen_aug_equ_epoch5.h5' 

#Model-RECA-CXP
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/chexpert-models/bfantasykuo_gmail_com/model_aug_v7-ssim_epoch5.h5' 

#Model-PHO-CXP
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/chexpert-models/bfantasykuo_gmail_com/model_chexpert_photo_epoch50.h5' 

#Model-TRA-CXP
# PRETRAINED_KERAS_MODEL = 'gs://hst-953-2019-shared-files/bfantasykuo_gmail_com/model_CXP_transfer_test.h5' 



In [ ]:
#MIMIC-CXR
# VALID_TFRECORDS = 'gs://mimic_cxr_derived/tfrecords/valid/frontal*' 

#Photo-MIMIC
# VALID_TFRECORDS = 'gs://hst-953-2019-shared-files/mimic-cxr-validationdata/bfantasykuo_gmail_com/Smartphone_frontal*'  

#CheXpert CXR
# VALID_TFRECORDS = 'gs://hst-953-2019-shared-files/chexpert/test_equ.tfrecord'  

#Photo-CheXpert
VALID_TFRECORDS = 'gs://hst-953-2019-shared-files/chexpert/test_photo.tfrecord'  

#Photo-CXP-DEV
#VALID_TFRECORDS = 'gs://hst-953-2019-shared-files/chexpert/test202.tfrecord'  
#VALID_TFRECORDS = 'gs://hst-953-2019-shared-files/chexpert/test202_photo_1.tfrecord'  

#Photo-CXP-MED
# VALID_TFRECORDS = 'gs://hst-953-2019-shared-files/chexpert/test_MD.tfrecord'  


In [ ]:
from __future__ import division
from __future__ import print_function
import datetime
import os
import tensorflow as tf
import multiprocessing
from enum import Enum
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics
import numpy as np
import subprocess
import re

try:
  from google.colab import auth
  IN_COLAB = True
  auth.authenticate_user()
except ImportError:
  IN_COLAB = False

account = subprocess.check_output(
    ['gcloud', 'config', 'list', 'account', '--format',
     'value(core.account)']).decode().strip()
MY_DIRECTORY = re.sub(r'[^\w]', '_', account)[:128]

%config InlineBackend.figure_format = 'svg'

In [ ]:
tf.enable_eager_execution()

*For the first time, run all the codes. After that, just run step 1 and 2.*

In [ ]:
#Iamge to be removed
Remove_subject=[10599550,11285576,12064623,16337160,16876554,17115211,17564874,18591791]
Remove_study=[4,9,5,1,2,2,2,5]
Remove_image=[1,1,1,1,1,1,2,1]

In [ ]:
class Labels(Enum):
  no_finding = 0
  enlarged_cardiomediastinum = 1
  cardiomegaly = 2
  airspace_opacity = 3
  lung_lesion = 4
  edema = 5
  consolidation = 6
  pneumonia = 7
  atelectasis = 8
  pneumothorax = 9
  pleural_effusion = 10
  pleural_other = 11
  fracture = 12
  support_devices = 13


class LabelValues(Enum):
  not_mentioned = 0
  negative = 1
  uncertain = 2
  positive = 3


class Views(Enum):
  frontal = 0
  lateral = 1
  other = 2


class Datasets(Enum):
  train = 0
  valid = 1

In [ ]:
#@title Input pipeline parameters {run: "auto"}
BATCH_SIZE = 1  #@param {type: "integer"}
NUM_EPOCHS = 2  #@param {type: "integer"}
U_VALUE = 0  #@param {type:"slider", min:0, max:1, step:0.01}
W_VALUE = 1  #@param {type:"slider", min:0, max:1, step:0.01}

In [ ]:
# label -> probability table: 0 -> 0, 1 -> 0, 2 -> u, 3 -> 1
probabs_lookup = tf.constant([0.0, 0.0, U_VALUE, 1.0])
# label -> weight table: 0 -> 1, 1 -> 1, 2 -> w, 3 -> 1
weights_lookup = tf.constant([1.0, 1.0, W_VALUE, 1.0])

In [ ]:
feature_description = {'jpg_bytes': tf.io.FixedLenFeature([], tf.string),'patient': tf.io.FixedLenFeature([], tf.int64),'study': tf.io.FixedLenFeature([], tf.int64),'image': tf.io.FixedLenFeature([], tf.int64)}
for l in Labels:
  feature_description[l.name] = tf.io.FixedLenFeature([], tf.int64)
# The height, width, and number of channels of the input images
INPUT_HWC = (320, 320, 1)

def parse_function(example):
  """Convert a TFExample from a TFRecord into an input and its true label.

    Args:
      example (tf.train.Example): A training example read from a TFRecord.

    Returns:
      Tuple[tf.Tensor, tf.Tensor]: The X-ray image and its labels. The labels
        are represented as two stacked arrays. One array is the probability
        that this label exists in the image, the other is how much weight this
        label should have when training the model.
    """
  parsed = tf.io.parse_single_example(example, feature_description)
  # Turn the JPEG data into a matrix of pixel intensities
  image = tf.io.decode_jpeg(parsed['jpg_bytes'], channels=1)
  # Give the image a definite size, which is needed by TPUs
  image = tf.reshape(image, INPUT_HWC)
  # Normalize the pixel values to be between 0 and 1
  scaled_image = (1.0 / 255.0) * tf.cast(image, tf.float32)
  # Combine the labels into an array
  labels = tf.stack([parsed[l.name] for l in Labels], axis=0)
  # Convert the labels into probabilities and weights using lookup tables.
  

  probs = tf.gather(probabs_lookup, labels)
  weights = tf.gather(weights_lookup, labels)
  # Return the input to the model and the true labels
  return scaled_image, tf.stack([probs, weights], axis=0)

def parse_image_name(example):

  parsed = tf.io.parse_single_example(example, feature_description)

  subject_num = tf.stack(parsed['patient'], axis=0)
  study_num = tf.stack(parsed['study'], axis=0)
  image_num = tf.stack(parsed['image'], axis=0)

  return subject_num, study_num, image_num

def parse_original_label(example):

  parsed = tf.io.parse_single_example(example, feature_description)
  labels = tf.stack([parsed[l.name] for l in Labels], axis=0)

  return labels

def get_dataset(valid=False):
  """Construct a pipeline for loading the data.

    Args:
      valid (bool): If this is True, use the validation dataset instead of the
        training dataset.

    Returns:
      tf.data.Dataset: A dataset loading pipeline ready for training.
    """
  n_cpu = multiprocessing.cpu_count()
  tf_records = VALID_TFRECORDS if valid else TRAIN_TFRECORDS
  dataset = tf.data.TFRecordDataset(
      tf.io.gfile.glob(tf_records),
      buffer_size=16 * 1024 * 1024,
      num_parallel_reads=n_cpu)
  if not valid:
    dataset = dataset.shuffle(256)
    dataset = dataset.repeat()
  dataset = dataset.map(parse_function, num_parallel_calls=n_cpu)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
#@title Accelerators {run: "auto"}
ACCELERATOR_TYPE = 'Multi-GPU'  #@param ["Single/Multi-TPU", "Single-GPU", "Multi-GPU", "CPU"] {type: "string"}

In [ ]:
if ACCELERATOR_TYPE == 'Single/Multi-TPU':
  if IN_COLAB:
    tpu_name = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  else:
    tpu_name = os.environ['TPU_NAME']
  resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=tpu_name)
  tf.contrib.distribute.initialize_tpu_system(resolver)
  strategy = tf.contrib.distribute.TPUStrategy(resolver, steps_per_run=100)
elif ACCELERATOR_TYPE == 'Multi-GPU':
  strategy = tf.distribute.MirroredStrategy()
else:
  strategy = tf.distribute.get_strategy()  # Default strategy

In [ ]:
def weighted_binary_crossentropy(prob_weight_y_true, y_pred):
  """Binary cross-entropy loss function with per-sample weights."""
  prob_weight_y_true = tf.reshape(prob_weight_y_true, (-1, 2, len(Labels)))
  # Unpack the second output of our data pipeline into true probabilities and
  # weights for each label.
  probs = prob_weight_y_true[:, 0]
  weights = prob_weight_y_true[:, 1]
  return tf.compat.v1.losses.sigmoid_cross_entropy(
      probs,
      y_pred,
      weights,
      reduction=tf.compat.v1.losses.Reduction.SUM_OVER_BATCH_SIZE)

In [ ]:
# with strategy.scope():
#   base_model = tf.keras.applications.densenet.DenseNet121(
#       include_top=False, weights=None, input_shape=INPUT_HWC, pooling='max')

#   predictions = tf.keras.layers.Dense(
#       len(Labels), activation='linear')(
#           base_model.output)

#   model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

  # model.compile(
  #     optimizer=tf.train.AdamOptimizer(),
  #     loss=weighted_binary_crossentropy,
  # )

In [ ]:
def download_from_gcs(gcs_path, local_path):
  with tf.io.gfile.GFile(gcs_path, 'rb') as gcs_file:
    with tf.io.gfile.GFile(local_path, 'wb') as local_file:
      local_file.write(gcs_file.read())

In [ ]:
def extract_dataset_label(valid=False):    
  n_cpu = multiprocessing.cpu_count()
  tf_records = VALID_TFRECORDS if valid else TRAIN_TFRECORDS
  Data_size = N_VALID if valid else N_TRAIN
  dataset = tf.data.TFRecordDataset(
      tf.io.gfile.glob(tf_records),
      buffer_size=16 * 1024 * 1024,
      num_parallel_reads=n_cpu)          
  dataset = dataset.map(parse_function, num_parallel_calls=n_cpu)  
  dataset = dataset.batch(Data_size, drop_remainder=True)  
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset
  
def extract_dataset_name(valid=False):    
  n_cpu = multiprocessing.cpu_count()
  tf_records = VALID_TFRECORDS if valid else TRAIN_TFRECORDS
  Data_size = N_VALID if valid else N_TRAIN
  dataset = tf.data.TFRecordDataset(
      tf.io.gfile.glob(tf_records),
      buffer_size=16 * 1024 * 1024,
      num_parallel_reads=n_cpu)          
  dataset = dataset.map(parse_image_name, num_parallel_calls=n_cpu)  
  dataset = dataset.batch(Data_size, drop_remainder=True)  
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

def extract_original_label(valid=False):    
  n_cpu = multiprocessing.cpu_count()
  tf_records = VALID_TFRECORDS if valid else TRAIN_TFRECORDS
  Data_size = N_VALID if valid else N_TRAIN
  dataset = tf.data.TFRecordDataset(
      tf.io.gfile.glob(tf_records),
      buffer_size=16 * 1024 * 1024,
      num_parallel_reads=n_cpu)          
  dataset = dataset.map(parse_original_label, num_parallel_calls=n_cpu)  
  dataset = dataset.batch(Data_size, drop_remainder=True)  
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
download_from_gcs(PRETRAINED_KERAS_MODEL, 'pretrained_model.h5')
with strategy.scope():
  model = tf.keras.models.load_model('pretrained_model.h5', compile=False)
  model.compile(
      optimizer=tf.train.AdamOptimizer(), loss=weighted_binary_crossentropy)

In [ ]:
#All pictures
example_predictions = model.predict(get_dataset(valid=True)) #, steps=3
print('shape: {}, min: {}, max: {}'.format(example_predictions.shape,
                                           example_predictions.min(),
                                           example_predictions.max()))

shape: (1337, 14), min: -18.4959716796875, max: 3.1816318035125732


In [ ]:
N_VALID = example_predictions.shape[0]

dataset=extract_dataset_label(valid=True)
for x,y in dataset:
  groundtruth=y
label3D=np.round(groundtruth.numpy())
label2D=label3D[:,0,:]

dataset_name = extract_dataset_name (valid=True)
for p,s,i in dataset_name:
  patient=p
  study=s
  image=i
dataset_label=extract_original_label(valid=True)
for p in dataset_label:
  original_label=p

In [ ]:
true_all = groundtruth[:,0,:].numpy().copy()

In [ ]:
#Create remove list according to the indices of images

remove_list = [0] * len(Remove_subject) 
no_remove = 0

for i in range(0,len(Remove_subject)):
  result1 = np.where(patient.numpy() == Remove_subject[i] )
  result1 = result1[0]
  ff=study.numpy()
  gg=ff[result1]
  result2 = np.where(gg == Remove_study[i])
  result2 = result2[0]
  hh=image.numpy()
  ii=hh[result1[result2]]
  result3 = np.where(ii== Remove_image[i])  
  result3 = result3[0]

  if (len(result1)==0 and len(result2)==0 and len(result3)==0):
    print('No matched images')
    no_remove = 1
    break
  remove_list[i]=result1[result2[result3]][0]


No matched images


In [ ]:
# # print all results 
# ss=0
# for label in range(14): 

#   true = groundtruth[:,0,label].numpy()
#   predicted = example_predictions[:,label]

#   if(no_remove):
#     true_tmp=true
#     predicted_tmp=predicted
#   else:
#     true_tmp=true[~np.in1d(range(len(true)),remove_list)]
#     predicted_tmp=predicted[~np.in1d(range(len(predicted)),remove_list)]


#   #precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true, predicted)
#   #average_precision = sklearn.metrics.average_precision_score(true, predicted)
  
#   #fpr, tpr, thresholds = sklearn.metrics.roc_curve(true, predicted)
#   if(len(np.where(true_tmp==1)[0])>1):
#     roc_auc = sklearn.metrics.roc_auc_score(true_tmp, predicted_tmp)
    
#     precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true_tmp, predicted_tmp)
#     average_precision = sklearn.metrics.average_precision_score(true_tmp, predicted_tmp)



#     fpr, tpr, thresholds = sklearn.metrics.roc_curve(true_tmp, predicted_tmp)
#     optimal_idx = np.argmax(tpr - fpr)
#     optimal_threshold = thresholds[optimal_idx]
#     predicted_class = np.zeros(len(predicted_tmp))
#     predicted_class[predicted_tmp > optimal_threshold] = 1

#     acc = sklearn.metrics.accuracy_score(true_tmp, predicted_class)
#     f1 = sklearn.metrics.f1_score(true_tmp, predicted_class)

#     tn, fp, fn, tp = sklearn.metrics.confusion_matrix(true_tmp, predicted_class).ravel()
#     specificity = tn / (tn + fp)
#     sensitivity = tp / (tp + fn) # recall
#     PPV = tp / (tp + fp) #precision
#     NPV = tn / (tn + fn) #NPV


#     print('[' + Labels(label).name + ']')
#     print(f'AUC: {roc_auc:.2f}')
#     print(f'Sensitivity: {sensitivity:.2f}')
#     print(f'Specificity: {specificity:.2f}')
#     print(f'PPV: {PPV:.2f}')
#     print(f'NPV: {NPV:.2f}')    
#     print(f'F1-score: {f1:.2f}')
#     print(f'Accuracy: {acc:.2f}')
    
    
#     ss = len(np.where(true_tmp==1)[0]) + ss
#     #print(average_precision)
#   else:
#     print('No cases')
#   #print('N=' + str(len(np.where(true_tmp==1)[0])))

In [ ]:
import statistics
# use min((1-tpr)^2+ (fpr) ^2 )
ss=0

bootstrapped_auc_all= []

for label in range(14): 

  true = groundtruth[:,0,label].numpy()
  predicted = example_predictions[:,label]

  if(no_remove):
    true_tmp=true
    predicted_tmp=predicted
  else:
    true_tmp=true[~np.in1d(range(len(true)),remove_list)]
    predicted_tmp=predicted[~np.in1d(range(len(predicted)),remove_list)]


  #precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true, predicted)
  #average_precision = sklearn.metrics.average_precision_score(true, predicted)
  
  #fpr, tpr, thresholds = sklearn.metrics.roc_curve(true, predicted)
  if(len(np.where(true_tmp==1)[0])>1):
    
    
    # precision, recall, thresholds = sklearn.metrics.precision_recall_curve(true_tmp, predicted_tmp)
    # average_precision = sklearn.metrics.average_precision_score(true_tmp, predicted_tmp)

    fpr, tpr, thresholds = sklearn.metrics.roc_curve(true_tmp, predicted_tmp)
    optimal_idx = np.argmin((1-tpr)**2+ (fpr)**2 )
    optimal_threshold = thresholds[optimal_idx]
    predicted_class = np.zeros(len(predicted_tmp))
    predicted_class[predicted_tmp > optimal_threshold] = 1
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(true_tmp, predicted_class).ravel()

    roc_auc_origin = sklearn.metrics.roc_auc_score(true_tmp, predicted_tmp)
    specificity_origin = tn / (tn + fp)
    sensitivity_origin = tp / (tp + fn) # recall
    PPV_origin = tp / (tp + fp) #precision
    NPV_origin = tn / (tn + fn) #NPV
    f1_origin = 2 * ((tp / (tp + fp)) * (tp / (tp + fn)) )/ ((tp / (tp + fp)) + (tp / (tp + fn)))
    acc_origin = (tp + tn) / (tn + fp + fn + tp)

    n_bootstraps = 1000
    rng_seed = 47  # control reproducibility
    bootstrapped_auc = []
    bootstrapped_specificity = []
    bootstrapped_sensitivity = []
    bootstrapped_PPV = []
    bootstrapped_NPV = []
    bootstrapped_f1 = []
    bootstrapped_acc = []

    rng = np.random.RandomState(rng_seed)
    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(predicted_tmp), len(predicted_tmp))
        while len(np.unique(true_tmp[indices])) < 2:
            # We need at least one positive and one negative sample for ROC AUC
            # to be defined: reject the sample
            indices = rng.randint(0, len(predicted_tmp), len(predicted_tmp))


        fpr, tpr, thresholds = sklearn.metrics.roc_curve(true_tmp[indices], predicted_tmp[indices])
        optimal_idx = np.argmin((1-tpr)**2+ (fpr)**2 )
        optimal_threshold = thresholds[optimal_idx]
        predicted_class = np.zeros(len(predicted_tmp))
        predicted_class[predicted_tmp > optimal_threshold] = 1
        tn, fp, fn, tp = sklearn.metrics.confusion_matrix(true_tmp[indices], predicted_class[indices]).ravel()

        auc = sklearn.metrics.roc_auc_score(true_tmp[indices], predicted_tmp[indices])
        specificity = tn / (tn + fp)
        sensitivity = tp / (tp + fn) # recall
        PPV = tp / (tp + fp) #precision
        NPV = tn / (tn + fn) #NPV
        # f1 = 2 * ((tp / (tp + fp)) * (tp / (tp + fn)) )/ ((tp / (tp + fp)) + (tp / (tp + fn)))
        f1 = sklearn.metrics.f1_score(true_tmp[indices], predicted_class[indices])
        acc = (tp + tn) / (tn + fp + fn + tp)

        bootstrapped_auc.append(auc)
        bootstrapped_specificity.append(specificity)
        bootstrapped_sensitivity.append(sensitivity)
        bootstrapped_PPV.append(PPV)
        bootstrapped_NPV.append(NPV)
        bootstrapped_f1.append(f1)
        bootstrapped_acc.append(acc)


    bootstrapped_auc_all.append(bootstrapped_auc)

        #print("Bootstrap #{} ROC area: {:0.3f}".format(i + 1, score))

    # print('[' + Labels(label).name + ']')
    print("{:0.4f} {:0.4f}".format(statistics.mean(bootstrapped_auc), statistics.stdev(bootstrapped_auc)))      
    print("{:0.2f} {:0.2f}".format(statistics.mean(bootstrapped_sensitivity)*100, statistics.stdev(bootstrapped_sensitivity)*100))
    print("{:0.2f} {:0.2f}".format(statistics.mean(bootstrapped_specificity)*100, statistics.stdev(bootstrapped_specificity)*100)) 
    # print("{:0.2f} {:0.2f}".format(statistics.mean(bootstrapped_PPV)*100, statistics.stdev(bootstrapped_PPV)*100))
    # print("{:0.2f} {:0.2f}".format(statistics.mean(bootstrapped_NPV)*100, statistics.stdev(bootstrapped_NPV)*100))
    print("{:0.4f} {:0.4f}".format(statistics.mean(bootstrapped_f1), statistics.stdev(bootstrapped_f1)))
    print("{:0.2f} {:0.2f}".format(statistics.mean(bootstrapped_acc)*100, statistics.stdev(bootstrapped_acc)*100))

    # print('[' + Labels(label).name + ']')
    # print(f'AUC: {roc_auc:.2f}')
    # print(f'Sensitivity: {sensitivity:.2f}')
    # print(f'Specificity: {specificity:.2f}')
    # print(f'PPV: {PPV:.2f}')
    # print(f'NPV: {NPV:.2f}')    
    # print(f'F1-score: {f1:.2f}')
    # print(f'Accuracy: {acc:.2f}')

    ss = len(np.where(true_tmp==1)[0]) + ss
    #print(average_precision)
  else:
    print('No cases')
  #print('N=' + str(len(np.where(true_tmp==1)[0])))

0.7988 0.0154
76.30 3.12
72.09 2.28
0.4674 0.0239
72.75 1.80
0.6647 0.0321
65.11 6.66
60.71 6.24
0.1474 0.0202
60.94 5.69
0.7343 0.0252
67.35 6.08
69.75 6.71
0.3060 0.0324
69.52 5.59
0.6919 0.0144
67.09 2.51
62.15 2.27
0.6112 0.0163
64.23 1.28
0.6177 0.0436
55.96 6.23
65.81 6.76
0.1194 0.0231
65.40 6.39
0.7355 0.0158
68.46 3.68
67.17 3.48
0.4986 0.0208
67.47 2.16
0.7229 0.0334
63.40 5.90
67.62 6.07
0.1421 0.0256
67.45 5.72
0.6109 0.0603
58.51 9.12
59.48 6.56
0.0502 0.0132
59.47 6.41
0.6125 0.0195
58.40 5.14
58.68 5.17
0.3176 0.0199
58.63 3.70
0.6347 0.0388
58.18 5.76
65.22 4.49
0.1467 0.0216
64.86 4.18
0.8251 0.0122
76.35 2.72
74.05 2.62
0.6327 0.0184
74.71 1.56
0.6265 0.0958
44.84 15.35
65.09 14.59
0.0193 0.0110
64.96 14.46
0.5637 0.0387
62.06 6.70
54.50 4.51
0.1016 0.0161
54.81 4.22
0.6631 0.0147
60.84 4.02
63.24 4.05
0.6053 0.0200
62.09 1.30


In [ ]:
bootstrapped_auc_all_np = np.array(bootstrapped_auc_all[:])

In [ ]:
mm =np.mean(bootstrapped_auc_all_np, axis = 0)

In [ ]:
bootstrapped_auc_all_np = np.vstack((bootstrapped_auc_all_np, mm))

In [ ]:
bootstrapped_auc_all_np.shape

(15, 1000)

In [ ]:
np.savetxt("foo.csv", bootstrapped_auc_all_np, delimiter=",")

In [ ]:
pp =np.mean(bootstrapped_auc_all_np, axis = 1)

In [ ]:
print(pp)

[0.79880577 0.66468846 0.73427415 0.69192676 0.61770357 0.73553514
 0.72285007 0.61087497 0.61248749 0.6346582  0.82513549 0.62646938
 0.56373729 0.66312147 0.67873344]
